In [1]:
# Import main packages
import sys, os
sys.path.insert(1, os.path.abspath('..') )
import ECOdiagnostics as eco
from ECOdiagnostics.utils import config_parser
import xarray as xr
import xgcm
import numpy as np

# Import matplotlib for plotting and functions to override default values
import matplotlib.pyplot as plt
from ECOanalysis import update_plotstyle, cmap_OB
cmap_OB=cmap_OB(); update_plotstyle()

In [ ]:
exp = 'EXP00_5d50'
path_to_dataset = 'data/'+exp # Here chosen as relative path, adjust if notebook is executed outside of 
file_name = exp+'_proc*'       # Import all files with this prefix
ds = xr.open_mfdataset(path_to_dataset+file_name, decode_times=False, parallel=True, data_vars='minimal') 

# get configuration from processing for matching parameters
kwargs_proc, kwargs_pre, kwargs_sim = config_parser(config_path='../ECOprocessing/Configs/'+exp, sub_config='sub_test')

# define xgcm configuration based on dataset metrics
_metrics = {('X',): ['e1tm', 'e1um', 'e1vm', 'e1fm'],
     ('Y',): ['e2tm', 'e2um', 'e2vm', 'e2fm'],
     ('Z',): ['e3tm', 'e3um', 'e3vm', 'e3wm']}

grid = xgcm.Grid(ds, metrics=_metrics, periodic=False) # define xgcm grid
grid_ops = eco.Grid_ops(grid, maskargs={'mask':'nan'}) # define gridoperations. With {'mask':'nan'} operations apply a nan-mask, replacing 0 by nan in data

# define coordinates to pass to diagnostics
_coords = {'X': ds.glamt,
           'Y': ds.gphit, 
           'Z': ds.depth_1d}

properties= eco.Properties(grid_ops, _coords, eos_properties=kwargs_sim['eos'])
energetics= eco.Energetics(grid_ops, properties)
energetics_trend= eco.Energetics_trends(grid_ops,properties,
                                             '../ECOdiagnostics/diagnostics/defaults/processes.ini')